###What is REFEED RAG?
REFEED RAG improves results without retraining by:

- Observing which retrieved results are relevant (clicks, corrections, thumbs up/down, etc.)

- Using this feedback to re-rank future retrievals.

- Sometimes boosting previously successful chunks when similar questions appear again.

- We’re not changing the model, but how you retrieve and rank context.

Intelligent Interview Assistant for HR
* Uses internal documents (policies, evaluation rubrics, guides).

* Retrieves info to assist in interviews.

* Learns over time from HR’s feedback (e.g., “This is outdated”).

* Improves future retrievals by re-ranking based on history.

In [ ]:
# ================== Install Dependencies ==================
!pip install -q langchain langchain-core langchain-community langchain-groq sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# ================== Imports ==================
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser
import os
import json

In [ ]:
# ================== Load HR Docs ==================
from google.colab import drive
# drive.mount('/content/drive')

# Load PDFs from HR_Policies folder
hr_folder = "/content/Human-Resources-Policy.pdf"
pdfs = [hr_folder]#os.path.join(hr_folder, file) for file in os.listdir(hr_folder) if file.endswith(".pdf")]

docs = []
for pdf in pdfs:
    loader = PyPDFLoader(pdf)
    docs.extend(loader.load_and_split())

# Split into chunks
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Total HR Chunks: {len(chunks)}")

Total HR Chunks: 31


In [ ]:
# ================== Embed and Store in FAISS ==================
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

/tmp/ipython-input-4-4251238885.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ================== LLM SETUP ==================
from google.colab import userdata

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=userdata.get("GROQ_API_KEY")
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a69ec1913d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a69c00d3cd0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# ================== Prompt Template ==================
hr_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent HR assistant. Use company policy context to answer interview-related questions."),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])
hr_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an intelligent HR assistant. Use company policy context to answer interview-related questions.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Context:\n{context}\n\nQuestion: {question}'), additional_kwargs={})])

In [ ]:
# ================== REFEED Feedback Store (JSON-Based) ==================
# Store feedback as a local dict or JSON file (in real-world: use vectorstore metadata or a DB)
feedback_log_path = "/content/hr_feedback_log.json"

# Initialize feedback log if not exists
if not os.path.exists(feedback_log_path):
    with open(feedback_log_path, "w") as f:
        json.dump({}, f)

# Load feedback history
def load_feedback():
    with open(feedback_log_path, "r") as f:
        return json.load(f)

# Save updated feedback
def save_feedback(feedback_data):
    with open(feedback_log_path, "w") as f:
        json.dump(feedback_data, f, indent=2)

In [ ]:
# ================== Retrieval + Reranking Logic with Feedback Boost ==================
def rerank_documents_with_feedback(question, docs):
    feedback = load_feedback()
    reranked = []

    for doc in docs:
        text = doc.page_content
        score = 1.0

        # Boost if this doc had positive feedback before for similar question
        for past_q, entries in feedback.items():
            if past_q.lower() in question.lower():  # Simple match, can use semantic sim
                for e in entries:
                    if e["text"] in text:
                        score += e["score"]  # 1 for positive, -1 for negative

        reranked.append((score, doc))

    reranked.sort(key=lambda x: x[0], reverse=True)
    return [doc for score, doc in reranked]

In [ ]:
# ================== Build the REFEED RAG Chain ==================
def retrieve_with_feedback(inputs):
    question = inputs["question"]
    retrieved = retriever.get_relevant_documents(question)
    reranked_docs = rerank_documents_with_feedback(question, retrieved)
    context = "\n\n".join([doc.page_content for doc in reranked_docs])
    return {"context": context, "question": question}

refeed_chain = (
    RunnableLambda(retrieve_with_feedback)
    | hr_prompt
    | llm
)
refeed_chain

RunnableLambda(retrieve_with_feedback)
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an intelligent HR assistant. Use company policy context to answer interview-related questions.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Context:\n{context}\n\nQuestion: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a69ec1913d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a69c00d3cd0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# ================== Ask Questions and Get Answers ==================
question = "What is our policy for remote work during probation?"
response = refeed_chain.invoke({"question": question})
print(response.content)

The provided policy document does not explicitly mention remote work during probation. However, it does outline the general principles and procedures for staff management, including the importance of effective communication, transparency, and fairness.

According to the policy, all staff members are expected to undergo a medical examination and participate in "pre-departure" training before commencing work. The policy also emphasizes the importance of a positive and open work environment, as well as the need for staff members to be aware of and respect the organization's Code of Conduct and policies.

Regarding remote work, the policy does not provide specific guidance on this topic during probation. However, it does mention that local staff are subject to local national regulations, and that the organization strives to create a positive work atmosphere by offering personal training opportunities and developing individual areas of interest and motivation.

In the absence of explicit gu

In [ ]:
# ================== Collect Feedback on Response ==================
# Sample feedback UI in notebook
user_feedback = input("Was the answer correct and useful? (yes/no): ").strip().lower()

if user_feedback in ["yes", "no"]:
    feedback = load_feedback()
    past = feedback.get(question, [])

    # Store top context snippet
    doc_snippet = response.content[:200]  # Top part of generated answer
    past.append({
        "text": doc_snippet,
        "score": 1 if user_feedback == "yes" else -1
    })

    feedback[question] = past
    save_feedback(feedback)
    print("Feedback recorded. Retrieval will improve next time.")

Was the answer correct and useful? (yes/no): no
Feedback recorded. Retrieval will improve next time.


Feature	Implemented<br>
Dynamic retrieval	<br>
Post-retrieval reranking	<br>
Feedback collection	<br>
Feedback persistence	<br>
Context-aware reweighting	<br>